In [215]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import math

In [216]:
# Zadanie 1
class DataProcessing:
    @staticmethod
    def ShuffleData(Dane): # Mieszanie losowo danych
        for i in range(len(Dane) - 1, 0 , -1):
            j = random.randint(0, i)
            Dane.iloc[i], Dane.iloc[j] = Dane.iloc[j], Dane.iloc[i]
        return Dane
    
    @staticmethod
    def splitData(Dane, x): # Podział na zbiór treningowy i walidacyjny
        if(x >= 100 or x <= 0):
            raise ValueError("Podano zły zakres!")
        split = int(len(Dane)*x*0.01)
        daneTrening = Dane.iloc[:split, :]
        daneTest = Dane.iloc[split:, :]
        return daneTrening, daneTest
    
    @staticmethod
    def normalizeData(Dane): # Normalizowanie danych
        Dane_ = Dane.copy()
        wartosci = Dane_.select_dtypes(exclude="object")
        kolumny = wartosci.columns.tolist()
        for kolumna in kolumny:
            dane = Dane_.loc[:, kolumna]
            maks = max(dane)
            minimum = min(dane)
            for rzad in range(0, len(Dane_), 1):
                wartosc = (Dane_.at[rzad, kolumna] - minimum) / (maks - minimum)
                Dane_.at[rzad, kolumna] = wartosc
        return Dane_

In [217]:
DataSet = pd.read_csv("C:\\Users\\User\\Desktop\\Systemy Sztucznej Inteligencji\\Kolokwium 12.04.2022\seeds.csv")
DataSet.head()

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [218]:
DataSet.describe()

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,14.918744,14.595829,0.870811,5.643151,3.265533,3.699217,5.420653,1.994975
std,2.919976,1.310445,0.023320,0.443593,0.378322,1.471102,0.492718,0.813382
min,10.590000,12.410000,0.808100,4.899000,2.630000,0.765100,4.519000,1.000000
25%,12.330000,13.470000,0.857100,5.267000,2.954500,2.570000,5.046000,1.000000
50%,14.430000,14.370000,0.873400,5.541000,3.245000,3.631000,5.228000,2.000000
75%,17.455000,15.805000,0.886800,6.002000,3.564500,4.799000,5.879000,3.000000
max,21.180000,17.250000,0.918300,6.675000,4.033000,8.315000,6.550000,3.000000


In [219]:
sns.set_palette('Reds')
sns.set_palette('husl')

In [ ]:
sns.pairplot(DataSet,
             hue = 'Type',
             markers = '+')

In [ ]:
sns.violinplot(x="Kernel.Groove", y="Type",
              data=DataSet, inner="quartile")

In [ ]:
Data_shuffled = DataProcessing.ShuffleData(DataSet) # Mieszanie danych
set_trening, set_walidacja = DataProcessing.splitData(Data_shuffled, 80) # Podział na dwa podzbiory
set_trening.head(5)

In [ ]:
print(DataProcessing.normalizeData(set_trening)) # Normalizacja

In [ ]:
# Zadanie 2
class NaiveBayes:
    @staticmethod
    def srednia(atrybut):
        return sum(atrybut) / len(atrybut)
    
    @staticmethod
    def odch_stand(atrybut):
        sr = NaiveBayes.srednia(atrybut)
        odchylenie = sum([(x - sr)**2 for x in atrybut]) / float(len(atrybut) - 1)
        return np.sqrt(odchylenie)
    
    @staticmethod
    def triangle(atrybut, srednia, odch_stand):
        if((atrybut > srednia + math.sqrt(6) * odch_stand) or (atrybut < srednia - math.sqrt(6) * odch_stand)):
            return 0
        elif(srednia > atrybut and atrybut >= srednia - math.sqrt(6) * odch_stand):
            return (atrybut-srednia / 6*odch_stand**2) + (1 / math.sqrt(6)*odch_stand)
        elif(atrybut > atrybut and atrybut <= srednia + math.sqrt(6)*odch_stand):
            return -(atrybut-srednia / 6*odch_stand**2) + (1 / math.sqrt(6)*odch_stand)

    
    @staticmethod
    def klasyfikator_Bayes(data, sample):
        P = {'1': 1/3, '2': 1/3, '3': 1/3}
        # Średnia i Odchylenie dla każdego atrybutu dla każdej klasy
        for klasa in ['1', '2', '3']:
            # Separacja na klasy, usunięcie ostatniej kolumny
            data_set = data[data['Type'] == int(klasa)]
            data_set = data_set.drop(['Type'], axis = 1)
            #print(klasa + ":\n", data_set)
            
            sample_index = 0 # indeks obecnie sprawdzanego atrybutu
            
            for atrybut in ['Area', 'Perimeter', 'Compactness', 'Kernel.Length', 'Kernel.Width', 'Asymmetry.Coeff', 'Kernel.Groove']:
                sr = NaiveBayes.srednia(data_set[atrybut])
                odchylenie = NaiveBayes.odch_stand(data_set[atrybut])
                P[klasa] = NaiveBayes.triangle(sample[sample_index], sr, odchylenie)
                sample_index += 1
        print(P)
        maks = max(P, key = P.get)
        print("Wynik klasyfikacji:\n", P,"\nUzyskany typ: ", maks)
        return maks
        

In [ ]:
#NaiveBayes.klasyfikator_Bayes(set_trening,[18.36, 16.52, 0.8452, 6.666, 3.485, 4.933, 6.448]) # Type 2

In [ ]:
# Zadanie 3
def find(arg, wages, names):
    data = []
    for i in range(len(arg)):
        result = 0
        for j in range(len(arg.iloc[i]) - 1):
            result += arg.iloc[i][j] * wages[j]
        data.append(result)
    return names[data.index(max(data))]

wagi = {"Area": 1, "Perimeter": 0.3, "Compactness": 0.1, "Kernel.Length": 0.5, "Kernel.Width": 0.5, "Asymmetry.Coeff": 0.5,
        "Kernel.Groove": 1}
wagi = list(wagi.values())

col_names = ['Area', 'Perimeter', 'Compactness', 'Kernel.Length', 'Kernel.Width', 'Asymmetry.Coeff', 'Kernel.Groove']

#print("Wynik na podstawie przedstawionych parametrów:", find(set_trening, wagi, col_names))

In [ ]:
# Zadanie 4
class KNN:
    @staticmethod
    def taxiMetric(x, y, m): # Wyznacz odległość między dwoma obiektami
        distance = 0
        for i in range(len(x)-1):
            distance += abs(x.iloc[i] - y.iloc[i])
            
        return distance
    
    def clustering(sample, data, m, k): # Predykcja na podstawie danych
        distances = []
        classes = {'1': 0, '2': 0, '3': 0}
        for i in data.index:
            distances.append([KNN.taxiMetric(sample, data.iloc[i], m), math.floor(data.iloc[i].Type)]) # Obliczanie odległości
        distances = sorted(distances) # Sortowanie odległości rosnąco                    
        
        for i in range(k):
            classes[str(distances[i][1])] += 1 # Głosowanie

        return max(classes, key=classes.get)
    
    def test(set_walidacja, set_trening, k): # Testowanie data-frame'ów, k - ilość sąsiadów (minimum 1, zalecane minimum 3, najlepiej nieparzyście)
        print ("Testuje k = " + str(k) +"..")
        correct = 0
        m = 2 # daje nam metrykę Euklidesową
        for sample in set_walidacja.iloc:
            AI_answer = KNN.clustering(sample, set_trening, m, k)
            if AI_answer == str(math.floor(sample.Type)):
                correct += 1

        accuracy = correct/len(set_walidacja) * 100
        print("Accuracy: ", round(accuracy,2), "Correct: ", correct)

In [ ]:
# Test dla k=4, dane nieznormalizowane
KNN.test(set_walidacja, set_trening, 4)

In [ ]:
# Test dla k=4, dane znormalizowane
Data_shuffled_norm = DataProcessing.normalizeData(Data_shuffled)
set_trening_norm, set_walidacja_norm = DataProcessing.splitData(Data_shuffled_norm, 80) # Podział na dwa podzbiory
#KNN.test(set_walidacja_norm, set_trening_norm, 4)